# Experimentation: Digitally Reconstructed Radiography (DRR)
---

[Gopalakrishnan and Golland, 2022](https://doi.org/10.1007/978-3-031-23179-7_1) presented a method for solving inverse problems in intraoperative imaging using fast auto-differentiable digitally reconstructed radiographs. In this notebook, we apply the method described in their work to solve a similar problem in our analysis.

`DiffDRR` is implemented as a custom PyTorch module.

In [1]:
import matplotlib.pyplot as plt
import torch

from diffdrr.data import load_example_ct
from diffdrr.drr import DRR
from diffdrr.visualization import plot_drr

## DRR Generation

In [2]:
# Read in the volume and get the isocenter
volume, spacing = load_example_ct()
bx, by, bz = torch.tensor(volume.shape) * torch.tensor(spacing) / 2
print(volume.shape, spacing, bx, by, bz)

# Initialize the DRR module for generating synthetic X-rays
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

(512, 512, 133) [0.703125, 0.703125, 2.5] tensor(180.) tensor(180.) tensor(166.2500)
cuda


In [3]:
drr = DRR(
    volume,      # The CT volume as a numpy array
    spacing,     # Voxel dimensions of the CT
    sdr=300.0,   # Source-to-detector radius (half of the source-to-detector distance)
    height=200,  # Height of the DRR (if width is not seperately provided, the generated image is square)
    delx=4.0,    # Pixel spacing (in mm)
).to(device)

In [4]:
# Set the camera pose with rotations (yaw, pitch, roll) and translations (x, y, z)
rotations = torch.tensor([[torch.pi, 0.0, torch.pi / 2]], device=device)
translations = torch.tensor([[bx, by, bz]], device=device)


In [5]:
img = drr(rotations, translations, parameterization="euler_angles", convention="ZYX")

In [ ]:
plot_drr(img, ticks=False)
plt.show()